In [ ]:
# t2_compstat.py  (polished layout: outside legend, dashed thresholds, clear annotations)
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
# Create output folders if they don't exist (for figures and CSV summaries)
os.makedirs("figures", exist_ok=True); os.makedirs("results", exist_ok=True)

# ---------- Baseline ----------
# Game primitives for T=2 comparative statics:
# p: prior that LEFT arm is good; s: success prob if the good arm is pulled; V: prize value.
# c0, r0: baseline thinking cost and breakthrough probability used for one-at-a-time sweeps.
p, s, V = 0.6, 0.7, 1.0
c0, r0  = 0.10, 0.40

# Colors for region shading in the plots (consistent palette across figures).
COLORS = {
    "think":   "#d1e7dd",  # green-ish
    "interior":"#cfe2ff",  # blue-ish
    "do":      "#fde2e4",  # pink-ish
}

def L_one_period_left(prior, sp_good, prize, r):
    """
    One-period continuation value L for the left arm under the model's information structure.
    Interprets 'thinking' as generating a breakthrough with prob r that reveals the good arm.
    If no breakthrough, you revert to the prior. Algebraically: L = s*V*[ r + (1 - r)*p ].
    This is the key scalar that pins down the T=2 interior mixing region.
    """
    return sp_good * prize * (r + (1 - r) * prior)

def xstar_do_prob(prior, sp_good, prize, think_cost, r):
    """
    Given primitives and (c, r), compute the symmetric mixed equilibrium at t=1 for T=2.
    Returns:
      x  = equilibrium probability of DO (pull immediately),
      1-x = 'Think frequency',
      region label in {"always-think","interior","always-do"},
      lower_c, upper_c = cost thresholds that delimit regions.
    Threshold logic:
      - If c <= 0.5*L - flat_do  -> 'always-think'
      - If c >= 1.0*L - flat_do  -> 'always-do'
      - Else interior: indifference pins x (linear in c / L).
    Notes:
      flat_do = expected one-shot payoff from DO with prior p (no information), i.e., p*s*V.
      L       = continuation value of information (defined above).
    """
    L = L_one_period_left(prior, sp_good, prize, r)
    flat_do = prior * sp_good * prize
    # Cost thresholds that separate pure/ mixed regions
    lower_c = 0.5 * L - flat_do
    upper_c = L - flat_do
    if think_cost <= lower_c:   # always-think region
        return 0.0, 1.0, "always-think", lower_c, upper_c
    if think_cost >= upper_c:   # always-do region
        return 1.0, 0.0, "always-do", lower_c, upper_c
    # Interior mixing: solve player indifference for x*, then clip to [0,1] for numerical safety.
    x = (2.0 * (flat_do + think_cost) / L) - 1.0
    x = float(np.clip(x, 0.0, 1.0))
    return x, 1.0 - x, "interior", lower_c, upper_c

# ---------- Plot 1: Think vs cost ----------
# Sweep c to show how the equilibrium Think frequency responds, holding r fixed at r0.
c_grid = np.linspace(0.0, 0.8, 201)
L_base = L_one_period_left(p, s, V, r0); flat_do = p * s * V
lower_c = 0.5 * L_base - flat_do
upper_c = L_base - flat_do
# Equilibrium Think frequency across c-grid
think_vals_c = [xstar_do_prob(p, s, V, c, r0)[1] for c in c_grid]
# Baseline marker at (c0, r0)
_, tf_c0, _, _, _ = xstar_do_prob(p, s, V, c0, r0)

fig, ax = plt.subplots(figsize=(7.2, 4.2))
ax.plot(c_grid, think_vals_c, label="Think frequency (1 - x*)")

# Shading the three regions in c-space using the thresholds (lower_c, upper_c)
if lower_c > 0.0:
    ax.axvspan(0.0, min(lower_c, 0.8), alpha=0.25, facecolor=COLORS["think"], label="Always Think")
left, right = max(0.0, lower_c), min(0.8, upper_c)
if left < right:
    ax.axvspan(left, right, alpha=0.25, facecolor=COLORS["interior"], label="Interior mix")
if upper_c < 0.8:
    ax.axvspan(max(0.0, upper_c), 0.8, alpha=0.25, facecolor=COLORS["do"], label="Always Do")

# Draw the dashed thresholds and annotate the upper boundary as c-dagger; place a baseline dot.
ax.axvline(x=lower_c, linestyle="--")
ax.axvline(x=upper_c, linestyle="--")
ax.annotate(f"c† ≈ {upper_c:.3f}", xy=(upper_c, 0.22), xytext=(upper_c+0.06, 0.34),
            arrowprops=dict(arrowstyle="->", lw=0.8), fontsize=9)
ax.scatter([c0], [tf_c0], zorder=3)

# Axes, labels, legend outside the plot area; save to disk
ax.set_xlim(0.0, 0.8); ax.set_ylim(0.0, 0.45)
ax.set_title("T=2: Period-1 Think Frequency vs Thinking Cost")
ax.set_xlabel("Thinking cost (c)"); ax.set_ylabel("Think frequency (1 - x*)")
# Legend outside right
ax.legend(loc="upper left", bbox_to_anchor=(1.02, 1.0), frameon=False)
fig.subplots_adjust(right=0.78)
fig.savefig("figures/t2_think_freq_vs_cost.png", dpi=200, bbox_inches="tight")
plt.close(fig)

# ---------- Plot 2: Think vs breakthrough r ----------
# Sweep r to show how the equilibrium Think frequency responds, holding c fixed at c0.
r_grid = np.linspace(0.01, 0.99, 199)
think_vals_r = [xstar_do_prob(p, s, V, c0, r)[1] for r in r_grid]

def r_boundary(mult):  # Solve flat_do + c0 = mult * L(r)
    """
    Closed-form boundary in r-space corresponding to a given multiplier on L(r):
      flat_do + c0 = mult * L(r)
    For mult=0.5 we obtain the lower cost boundary mapped into r (start of interior from 'always-think');
    for mult=1.0 we obtain the upper boundary mapped into r (end of interior into 'always-do').
    Returns the implied r* (may be outside [0,1], so callers check bounds before drawing).
    Derivation:
      L(r) = s*V*( r + (1 - r)*p ) = s*V*( p + r*(1 - p) ).
      Solve for r:
        flat_do + c0 = mult*s*V*( p + r*(1 - p) )
        r = [ flat_do + c0 - mult*s*V*p ] / [ mult*s*V*(1 - p) ].
    """
    A = s * V * (1 - p) * mult
    B = s * V * p * mult
    return (flat_do + c0 - B) / A

# r_upper is the 'always-do' boundary; r_lower is the 'always-think' boundary (names follow the c-plot).
r_lower, r_upper = r_boundary(0.5), r_boundary(1.0)
# Baseline marker at (c0, r0)
_, tf_r0, _, _, _ = xstar_do_prob(p, s, V, c0, r0)

fig, ax = plt.subplots(figsize=(7.2, 4.2))
ax.plot(r_grid, think_vals_r, label="Think frequency (1 - x*)")

# Region shading in r-space using r_upper (left boundary) and r_lower (right boundary).
# Left of r_upper -> Always Do; between -> Interior; beyond r_lower -> Always Think.
if 0.0 < r_upper < 1.0:
    ax.axvspan(0.0, r_upper, alpha=0.25, facecolor=COLORS["do"], label="Always Do")
else:
    ax.axvspan(0.0, 1.0, alpha=0.25, facecolor=COLORS["do"], label="Always Do")
left = max(0.0, r_upper); right = min(1.0, r_lower) if np.isfinite(r_lower) else 1.0
if left < right:
    ax.axvspan(left, right, alpha=0.25, facecolor=COLORS["interior"], label="Interior mix")
if np.isfinite(r_lower) and r_lower < 1.0:
    ax.axvspan(max(0.0, r_lower), 1.0, alpha=0.25, facecolor=COLORS["think"], label="Always Think")

# Dashed vertical lines at boundaries, annotate r-dagger (upper boundary), and baseline dot.
if 0.0 < r_upper < 1.0:
    ax.axvline(x=r_upper, linestyle="--")
    ax.annotate(f"r† ≈ {r_upper:.3f}", xy=(r_upper, 0.22), xytext=(r_upper+0.05, 0.34),
                arrowprops=dict(arrowstyle="->", lw=0.8), fontsize=9)
if np.isfinite(r_lower) and 0.0 < r_lower < 1.0:
    ax.axvline(x=r_lower, linestyle="--")
ax.scatter([r0], [tf_r0], zorder=3)

# Axes, labels, legend outside; save to disk
ax.set_xlim(0.0, 1.0); ax.set_ylim(0.0, 0.55)
ax.set_title("T=2: Period-1 Think Frequency vs Breakthrough Probability")
ax.set_xlabel("Breakthrough probability (r)"); ax.set_ylabel("Think frequency (1 - x*)")
ax.legend(loc="upper left", bbox_to_anchor=(1.02, 1.0), frameon=False)
fig.subplots_adjust(right=0.78)
fig.savefig("figures/t2_think_freq_vs_r.png", dpi=200, bbox_inches="tight")
plt.close(fig)

# ---------- Small CSV ----------
# Write a compact CSV benchmarking select points on each sweep:
#   - endpoints and thresholds in c (at r0),
#   - endpoints and thresholds in r (at c0).
rows = []
for (lab, c, r) in [
    ("c=0.0", 0.0, r0), ("c=lower", lower_c, r0), ("c=upper", upper_c, r0),
    ("r=0.01", c0, 0.01), ("r=lower", c0, r_lower), ("r=upper", c0, r_upper), ("r=0.99", c0, 0.99)
]:
    # Evaluate equilibrium and record both the DO prob x and Think frequency 1-x, plus the qualitative region.
    x, tf, reg, _, _ = xstar_do_prob(p, s, V, c, r)
    rows.append({"label": lab, "c": float(c), "r": float(r), "x_do": x, "think_freq": tf, "region": reg})
pd.DataFrame(rows).to_csv("results/t2_compstat_summary.csv", index=False)

print("Saved figures/t2_think_freq_vs_cost.png, figures/t2_think_freq_vs_r.png and results/t2_compstat_summary.csv")


Saved figures/t2_think_freq_vs_cost.png, figures/t2_think_freq_vs_r.png and results/t2_compstat_summary.csv
